In [8]:
import requests
import pandas as pd 
import datetime
from dotenv import load_dotenv
import os
from tqdm import tqdm
import utilities
import time

def get_lifi_data(timestamp):
    url = f'https://li.quest/v1/analytics/transfers?fromTimestamp={timestamp}&status=DONE'
    df = pd.DataFrame(requests.get(url).json()['transfers'])
    if len(df) > 0:
        df['timestamp'] = [i["timestamp"] for i in df.sending.values]
        df.index = pd.to_datetime(df['timestamp'], unit = 's')
        df.rename_axis('DATETIME', inplace=True)
        return df 
    else: 
        print('This', timestamp, 'has no transfers')


if __name__ == "__main__":

    GOOGLE_CREDENTIALS_PATH = os.getenv("GOOGLE_CREDENTIALS_PATH")
    BQ_DATASET_ID = os.getenv("CROSSCHAIN_DATASET_ID")
    BQ_TABLE_ID = os.getenv("LIFI_TABLE_ID")

    while True:
        
        start_timestamp = utilities.get_BQ(
            """
            SELECT MAX(TIMESTAMP) AS max_timestamp
            FROM `silken-mile-379810.crosschain_dataset.lifi_data`
            """, GOOGLE_CREDENTIALS_PATH)
    
        timestamp = int(start_timestamp.iloc[0])

        # Fetch data from lifi API
        df = get_lifi_data(timestamp)
        df = df.loc[df.timestamp > timestamp]

        # Load data into BigQuery
        if len(df) > 0:
            utilities.load_to_table(df, BQ_DATASET_ID, BQ_TABLE_ID, GOOGLE_CREDENTIALS_PATH )
        else: 
            pass

        time.sleep(10)

Data successfully loaded to table 'silken-mile-379810.crosschain_dataset.lifi_data'.


KeyboardInterrupt: 